# Forecast

In [34]:
# importing all necessary libraries

# pip install -r requirements.txt

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from datetime import datetime, timedelta
import statsmodels.api as sm
import gc
from pylab import rcParams
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import lightgbm as lgb
import calendar

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Load data

In [35]:
# Reading datasets

train_model = pd.read_csv('input/train_ready.csv')
test_model = pd.read_csv('input/test_ready.csv')

# Run model

In [36]:
# Train an XGBoost model

# Set Grid

param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Train an XGBoost model
    
xgb_model = XGBRegressor()

grid_search = GridSearchCV(estimator = xgb_model, param_grid = param_grid, cv=3, scoring = 'neg_mean_squared_error')
    
grid_search.fit(train_model.drop(['id', 'inventory_units'], axis=1), train_model['inventory_units'])

best_xgb_model = grid_search.best_estimator_

In [37]:
# Apply predictions
    
product_predictions = best_xgb_model.predict(test_model.drop(['id'], axis=1))

test_model['predictions'] = product_predictions

# Feature importances

In [38]:
imps = pd.Series(best_xgb_model.feature_importances_, index=train_model.drop(['id', 'inventory_units'], axis=1).columns)
imps = imps.reset_index()
imps.columns = ['predictor', 'importance']
imps['predictor_group'] = imps['predictor'].str.split('_').str[0]
imps = imps.groupby('predictor_group').sum().sort_values(by = "importance", ascending = False)
imps

,importance
predictor_group,
product,0.336787
inventory,0.305116
reporterhq,0.125128
month,0.065707
segment,0.042203
year,0.037709
prod,0.032479
sales,0.027595
season,0.022646


# Write output

In [39]:
# Aggregate at id level

fcst_data_output = test_model.groupby(['id']).sum().reset_index()

fcst_data_output.to_csv('output/fcst_results.csv', index = False)

# Output

In [40]:
# Submission

submission = fcst_data_output[['id', 'predictions']]

submission.columns = ['id', 'inventory_units']

submission.to_csv('submission/kaggle_submission.csv', index = False)

submission

,id,inventory_units
0,202319-105609,139.734741
1,202319-107583,423.016968
2,202319-10857,137.472198
3,202319-109557,137.988098
4,202319-112518,237.911804
...,...,...
1295,202331-92778,151.974899
1296,202331-93765,137.198441
1297,202331-94752,191.429779
1298,202331-9870,124.454887
